In [8]:
%matplotlib inline
import quantstats as qs
import yfinance as yf
# extend pandas functionality with metrics, etc.
qs.extend_pandas()

# fetch the daily returns for a stock
msft = yf.Ticker("MSFT")
hist = msft.history(period="max")
hist

# show sharpe ratio
# qs.stats.sharpe(stock)

# or using extend_pandas() :)
# stock.sharpe()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1986-03-13,0.055898,0.064119,0.055898,0.061378,1031788800,0.0,0.0
1986-03-14,0.061378,0.064667,0.061378,0.063570,308160000,0.0,0.0
1986-03-17,0.063570,0.065215,0.063570,0.064667,133171200,0.0,0.0
1986-03-18,0.064667,0.065215,0.062474,0.063022,67766400,0.0,0.0
1986-03-19,0.063022,0.063570,0.061378,0.061926,47894400,0.0,0.0
...,...,...,...,...,...,...,...
2022-02-01,310.410004,310.630005,305.130005,308.760010,40950400,0.0,0.0
2022-02-02,309.630005,315.119995,308.880005,313.459991,36636000,0.0,0.0
2022-02-03,309.489990,311.230011,299.959991,301.250000,43730000,0.0,0.0
